In [2]:
from classes import *
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from math import cos, sin, sqrt
from numba import jit
from GMM import *
from sklearn.decomposition import PCA

Resizing Masks: 100%|██████████| 48/48 [00:00<00:00, 6443.89it/s]


 (768, 1024, 3) -> (192, 256, 3)


In [3]:
#Initialize puzzle
puzzle = Puzzle(MATCH_IMGS)
pieces = np.array(puzzle.pieces)
kfold = KFold(6, True, 1)
heigt, width, _ = puzzle.pieces[0].image.shape

Generating Pieces: 48it [00:10,  4.49it/s]
/Library/Python/3.7/site-packages/sklearn/utils/validation.py:71: FutureWarning: Pass shuffle=True, random_state=1 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


In [4]:
test_params = np.array([ [2, 4, 6],
                         [2, 4, 6]] )

#Get test and train features
def get_DoG_train(pieces):
    foreground = []
    background = []
    for p in pieces:
        foreground.append(np.concatenate([p.RGB_foreground, p.DoG_foreground], axis = 1))
        background.append(np.concatenate([p.RGB_background, p.DoG_background], axis = 1 ))
    foreground = np.concatenate(foreground)
    background = np.concatenate(background)
    
    return foreground, background
  
def get_DoG_test(pieces):
    X_test = []
    msks = []
    for p in pieces:
        X_test.append(np.concatenate([p.features_RGB, p.features_DoG], axis = 1))
        msks.append(p.mask.flatten())
    return np.concatenate(X_test), np.concatenate(msks)


def get_RGB_train(pieces):
    foreground = []
    background = []
    for p in pieces:
        foreground.append(np.concatenate([p.RGB_foreground, p.DoG_foreground], axis = 1))
        background.append(np.concatenate([p.RGB_background, p.DoG_background], axis = 1 ))
    foreground = np.concatenate(foreground)
    background = np.concatenate(background)
    
    return foreground, background

    
def get_RGB_test(pieces):
    X_test = []
    msks = []
    for p in pieces:
        X_test.append(np.concatenate([p.features_RGB, p.features_DoG], axis = 1))
        msks.append(p.mask.flatten())
    return np.concatenate(X_test), np.concatenate(msks)

def rotate(arr):
    """
    Rotates claims between vehicles in fixed period
    """
    temp = arr[0]
    n = len(arr)
    for k in range(n-1):
        arr[k] = arr[k+1]
    arr[n-1] = temp
    return arr

#Initialize parameters to optimize
def permutations(data, n_cars, curr_period, n_periods, output = []):
    """ INPUT
        data: a day worth of fitments, fixed to periods
        n_cars: number of cars
        n_periods: Number of periods
        
        OUTPUT
        output: an array of all permuations of the day (should be n_cars^(n_periods - 1))
        """
    if curr_period == n_periods:
        output.append(np.array(data.tolist())[:, 0])
        return
    
    for i in range(n_cars):
        data[curr_period, :] = rotate(data[curr_period, :])
        permutations(data, n_cars, curr_period + 1, n_periods, output)
    return output

tests = np.array(permutations(test_params, 3, 0, 2))

In [5]:
def get_MR8_train(pieces):
    foreground = []
    background = []
    for p in pieces:
        foreground.append(p.MR8_foreground)
        background.append(p.MR8_background)
    foreground = np.concatenate(foreground)
    background = np.concatenate(background)
    
    return foreground, background

    
def get_MR8_test(pieces):
    X_test = []
    msks = []
    for p in pieces:
        X_test.append(p.features_MR8)
        msks.append(p.mask.flatten())
    return np.concatenate(X_test), np.concatenate(msks)

def get_MR8_train(pieces):
    foreground = []
    background = []
    for p in pieces:
        foreground.append(p.MR8_foreground)
        background.append(p.MR8_background)
    foreground = np.concatenate(foreground)
    background = np.concatenate(background)
    
    return foreground, background

def get_PCA_train(pieces):
    foreground = []
    background = []
    for p in pieces:
        foreground.append(p.PCAReduced_foreground)
        background.append(p.PCAReduced_background)
    foreground = np.concatenate(foreground)
    background = np.concatenate(background)
    
    return foreground, background

def get_PCA_test(pieces):
    X_test = []
    msks = []
    for p in pieces:
        X_test.append(p.features_PCAReduced)
        msks.append(p.mask.flatten())
    return np.concatenate(X_test), np.concatenate(msks)

In [6]:
#Plot probability map for sanity check
def plot_prob(height = 192, width = 256, p = None, k= 0 ):
    plt.imshow(p[k*height*width:(k+1)*height*width].reshape(height, width))

In [7]:
def run(foreground, background, X_test, y_test, n_f, n_b, threshold):
    """
    foreground: foreground pixels
    background: background pixels
    X_test: test features
    y_test: masks
    threshold: threshold
    """
    #Train models
    try:
        gmm_foreground = GMM(N = n_f)
        gmm_foreground.run(foreground)
        gmm_background = GMM(N = n_b)
        gmm_background.run(background)
        #Make prediction
    except:
        print("gmm did not run error")
        return -1
    l = len(foreground) / (len(foreground) + len(background))
    prob_f = gmm_foreground.predict(X_test)
    prob_b = gmm_background.predict(X_test)
    #Calculate probability ( Sometimes the pictures break when I use less clusters)
    prob = l*prob_f / (l*prob_f + (1-l)*prob_b )
    
        
    return roc_auc_score(prob > 0.5, y_test)

In [8]:
def run_tests(features_Train = None, features_Test = None,  threshold = 0.5, k = 6, pieces = None, HypeParam = None):
        """
        features_Train : Function to extract train features
        features_Test: Function to extract test features
        """
        kfold = KFold(k)
        mean_results = []
        for t in HypeParam:
            #get parameters
            n_f = int(t[0])
            n_b = int(t[1])
            results = []
            print(t)
            #Check on all 6-folds
            for train, test in kfold.split(pieces):
                foreground, background = features_Train(pieces[train])
                X, y = features_Test(pieces[test])
                score = run(foreground = foreground, background = background, X_test = X, y_test= y, n_f = n_f, n_b = n_b, threshold = threshold)  
            mean_results.append(np.mean(results))
            print( "Mean", np.mean(results))
        return np.amax(mean_results)

In [ ]:
print("RGB features")
score = run_tests(features_Train = get_RGB_train, features_Test = get_RGB_test,  threshold = 0.5, k = 6, pieces = pieces, HypeParam = tests)
print("Best RGB score: ", score)
print("RGB & DoG features")
score = run_tests(features_Train = get_DoG_train, features_Test = get_DoG_Train,  threshold = 0.5, k = 6, pieces = pieces, HypeParam = tests)
print("Best DoG score: ", score)
print("MR8 features")
score = run_tests(features_Train = get_MR8_train, features_Test = get_MR8_Train,  threshold = 0.5, k = 6, pieces = pieces, HypeParam = tests)
print("Best MR8 score: ", score)

RGB features
[4 4]


/Users/jamieburns/Library/Python/3.7/lib/python/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/jamieburns/Library/Python/3.7/lib/python/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Mean nan
[4 6]


/Users/jamieburns/Library/Python/3.7/lib/python/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/jamieburns/Library/Python/3.7/lib/python/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Mean nan
[4 2]
